In [1]:
import hashlib
import json
from heuristique.genetique import *
from objects.population import Population
from objects.solution import Solution
from script.utils import sort_task, decodage_gene
from objects.instance import Instance
from script.affichage import plot_gantt, net_voisinage
from script.data import loader 
import numpy as np
import networkx as nx 
import copy
import matplotlib.pyplot as plt
import random
from heuristique.heuristique_gloutonne import heuristique_gloutone
from heuristique.ordo_avec_liste import alloc_avec_liste, allocation_dispomachine_ASAP,decodage_bier
from script.utils import vecteur_bier
from script.recherche_voisin import *

data, optimum= loader('ft10')
meth = [inversion_all, decalage_all, permutation_by_2_all]


In [ ]:
pop = Population(**data)
main_genetique(pop, data, 25,100,780)


Géneration:  0
==========================Nouvel elite !==========================
ID: 1-d4e3//c46f/174d, Pere: 0-c46f-A&E , Mere: 0-174d-A&E , Cout: 1359, Proba: 1.6 %
Facteur demuta: 0.04
Géneration:  1
Géneration:  2
==========================Nouvel elite !==========================
ID: 3-d4de//7a94/5bd7, Pere: 2-7a94//010b/7aeb, Mere: 2-5bd7//7a70/7aeb, Cout: 1316, Proba: 1.6 %
Facteur demuta: 0.12000000000000002
Géneration:  3
==========================Nouvel elite !==========================
ID: 3-c117//d69f/5bd7, Pere: 2-d69f//7be7/d09b, Mere: 2-5bd7//7a70/7aeb, Cout: 1310, Proba: 1.6 %
Facteur demuta: 0.12000000000000002
Géneration:  4
==========================Nouvel elite !==========================
ID: 4-af6a//c117/d4de, Pere: 3-c117//d69f/5bd7, Mere: 3-d4de//7a94/5bd7, Cout: 1255, Proba: 1.6 %
Facteur demuta: 0.12000000000000002
Géneration:  5
Géneration:  6
==========================Nouvel elite !==========================
ID: 7-aec1//07fa/8fdf, Pere: 6-07fa//d4de/8fdf, Mer

In [ ]:
from script.recherche_voisin import exploration_voisinage, inversion_all, permutation_random

b=exploration_voisinage(instance , data)

In [ ]:
net_voisinage(b)


#
 

In [ ]:
compt = 0
minRND = 100000    
inst2 = Instance(**data)
totalSPT = heuristique_gloutone(inst2, verbose = 0, prio ="SPT" , rnd = 0)
minSPT = totalSPT

for i in range(100):
    inst = Instance(**data)
    totalRND = heuristique_gloutone(inst, verbose = 0, prio ="SPT" , rnd = 1)
    if totalRND < minRND :
        minRND = totalRND
    
    if totalRND > totalSPT:
        compt += 1
        
print("RND perd", compt,"/", len(range(100)))
print("RND gagne", len(range(100)) - compt,"/", len(range(100)))
print("Optimium", optimum)
print("minRND", minRND)
print("minSPT", minSPT)

In [15]:
l = ['ft']

for e in l:        
    with open('data/instances.json') as json_data:
        data = json.load(json_data)
        for i, f in enumerate(data):
            if f['name'][:2] == e:
                index = i
                nb_machine = data[index]['machines']
                nb_jobs = data[index]['jobs']
                optimum = data[index]['optimum']
                file = open("data/instances/" + f['name'], "r")
                tab = []
                for i, line in enumerate(file):
                    if e[:2]== 'ta':
                        if i > 0:
                            tab.append([int(e) for e in line.split()])
                    else:
                        if i > 4:
                            tab.append([int(e) for e in line.split()])
                
                datas, optimum = {"nb_machine":nb_machine, "nb_jobs":nb_jobs, "problem":tab}, optimum
                moy1= 0
                moy2 = 0
                li = [0, 0.2,0.5,1]
                for j in li:
                    moy1= 0
                    moy2 = 0
                    for i in range(100):
                        inststp = Instance(**datas)
                        heuristique_gloutone(inststp,'SPT',rnd=j)
    
                        moy1 += inststp.makeSpan
                        
                        instltp = Instance(**datas)
                        heuristique_gloutone(instltp,'LPT',rnd=j)
                        moy2 += instltp.makeSpan
    
                    print(f['name'] + ', rnd= ' +str(j),'&',moy1/100,'&',moy2/100,'&',optimum,'\\','\\')
                    print('\hline')

ft06, rnd= 0 & 90.0 & 79.0 & 55 \ \
\hline
ft06, rnd= 0.2 & 80.23 & 74.07 & 55 \ \
\hline
ft06, rnd= 0.5 & 71.85 & 69.89 & 55 \ \
\hline
ft06, rnd= 1 & 68.56 & 69.53 & 55 \ \
\hline
ft10, rnd= 0 & 1075.0 & 1299.0 & 930 \ \
\hline
ft10, rnd= 0.2 & 1150.35 & 1286.14 & 930 \ \
\hline
ft10, rnd= 0.5 & 1199.6 & 1275.38 & 930 \ \
\hline
ft10, rnd= 1 & 1227.35 & 1240.99 & 930 \ \
\hline
ft20, rnd= 0 & 1268.0 & 1633.0 & 1165 \ \
\hline
ft20, rnd= 0.2 & 1339.6 & 1598.89 & 1165 \ \
\hline
ft20, rnd= 0.5 & 1390.74 & 1581.68 & 1165 \ \
\hline
ft20, rnd= 1 & 1528.92 & 1527.58 & 1165 \ \
\hline


In [5]:
data, optimum= loader('ft06')
instltp = Instance(**data)
heuristique_gloutone(instltp, prio = 'SPT')
liste= vecteur_bier(instltp)
temps1 = time.time()
sol = Solution(data, liste, root= "None")
g = exploration_voisinage(sol, data, n = 1 , max_depth = 60)
temps2 = time.time()
print(temps2-temps1)

Root:  8c24 Voisinage:  c90f  ,makeSpan:  66 / 88  ,écart:  0.25
[1 1 1 0 5 0 2 3 5 2 4 5 0 3 0 1 2 3 5 3 1 4 0 3 2 3 5 2 5 4 4 0 4 2 4 1]
Root:  c90f Voisinage:  88f6  ,makeSpan:  61 / 66  ,écart:  0.07575757575757576
[1 1 1 0 5 0 2 3 5 2 4 5 0 3 0 1 2 3 5 4 1 4 0 3 2 3 5 2 5 4 4 0 3 2 4 1]
Root:  88f6 Voisinage:  372a  ,makeSpan:  59 / 61  ,écart:  0.03278688524590164
[2 1 1 0 5 0 2 3 5 2 4 5 0 3 0 1 2 3 5 4 1 4 0 3 1 3 5 2 5 4 4 0 3 2 4 1]
Root:  372a Voisinage:  7be6  ,makeSpan:  57 / 59  ,écart:  0.03389830508474576
[2 1 3 0 5 0 2 1 5 2 4 5 0 3 0 1 2 3 5 4 1 4 0 3 1 3 5 2 5 4 4 0 3 2 4 1]
Root:  7be6 Voisinage:  bae5  ,makeSpan:  56 / 57  ,écart:  0.017543859649122806
[2 1 3 0 5 0 2 1 5 2 4 5 3 3 0 1 2 3 5 4 1 4 0 3 1 0 5 2 5 4 4 0 3 2 4 1]
Root:  bae5 Voisinage:  00c3  ,makeSpan:  55 / 56  ,écart:  0.017857142857142856
[2 1 3 0 5 0 2 1 5 2 4 5 3 3 0 1 2 3 5 4 1 4 0 3 1 0 5 2 5 1 4 0 3 2 4 4]
Root:  00c3 Voisinage:  1ca0  ,makeSpan:  55 / 55  ,écart:  0.0
[2 1 3 0 5 0 2 1 5 2 4 5 

In [6]:
data, optimum= loader('ft10')
instltp = Instance(**data)
heuristique_gloutone(instltp, prio = 'SPT')
liste= vecteur_bier(instltp)
temps1 = time.time()
sol = Solution(data, liste, root= "None")
g = exploration_voisinage(sol, data, n = 1 , max_depth = 60)
temps2 = time.time()
print(temps2-temps1)

Root:  3b6d Voisinage:  d4c8  ,makeSpan:  1038 / 1074  ,écart:  0.0335195530726257
[0 4 6 7 4 1 5 4 0 4 6 6 8 1 5 3 5 4 7 4 5 0 6 8 3 6 1 4 6 4 6 0 5 7 8 3 4
 7 9 6 0 3 1 4 0 1 5 2 8 9 6 9 0 5 8 7 1 0 9 3 9 6 2 5 8 0 9 5 8 5 2 3 7 8
 2 0 9 7 1 9 3 8 2 9 1 3 2 7 7 8 2 2 7 3 1 3 9 1 2 2]
Root:  d4c8 Voisinage:  269b  ,makeSpan:  1028 / 1038  ,écart:  0.009633911368015413
[0 4 6 7 4 1 5 4 0 4 4 6 8 1 5 3 5 4 7 6 5 0 6 8 3 6 1 4 6 4 6 0 5 7 8 3 4
 7 9 6 0 3 1 4 0 1 5 2 8 9 6 9 0 5 8 7 1 0 9 3 9 6 2 5 8 0 9 5 8 5 2 3 7 8
 2 0 9 7 1 9 3 8 2 9 1 3 2 7 7 8 2 2 7 3 1 3 9 1 2 2]
Root:  269b Voisinage:  80b7  ,makeSpan:  1025 / 1028  ,écart:  0.0029182879377431907
[0 4 6 7 4 1 5 4 0 4 4 6 8 1 5 3 5 4 7 6 5 0 6 8 3 6 1 4 6 4 6 0 5 7 8 3 4
 7 9 6 0 3 1 4 0 1 5 2 8 9 6 9 0 5 8 7 1 0 9 3 9 6 2 5 8 0 9 5 8 5 2 3 7 8
 2 0 9 7 1 9 3 8 2 9 1 3 7 2 2 8 7 7 2 3 1 3 9 1 2 2]
Root:  80b7 Voisinage:  07e0  ,makeSpan:  1005 / 1025  ,écart:  0.01951219512195122
[0 4 6 7 4 1 5 4 0 4 4 6 8 1 5 3 5 4 7 6 5 0 6 8 3

In [9]:
data, optimum= loader('ft20')
instltp = Instance(**data)
heuristique_gloutone(instltp, prio = 'SPT')
liste= vecteur_bier(instltp)
temps1 = time.time()
sol = Solution(data, liste, root= "None")
g = exploration_voisinage(sol, data, n = 10 , max_depth = 60)
temps2 = time.time()
print(temps2-temps1)

Root:  17af Voisinage:  b50a  ,makeSpan:  1267 / 1267  ,écart:  0.0
[ 4 14 15  6 19  7  0  5 16 19  0  4  1  7  4 16  5  0  4  7  4 16  7  6
 19  0 14  7  8 18  6 19  0 11  6 12 19  1  8  6 18  5 11 10 17  1 12 14
 11 10 18  1 12  5 17 11  1 14 13 10  8 17 15 12 13  9  3 14 10 11  8  3
 12  5 15 10 16  2  9  3  8 17 15 18  9 13  3 16  2  9 18 15  3  2 17  9
 13  2 13  2]
Root:  17af Voisinage:  3db4  ,makeSpan:  1267 / 1267  ,écart:  0.0
[ 4 14 15  6 19  7  0  5 16 19  0  4  1  7  4 16  5  0  4  7  4 16  7  6
 19  0 14  7  8 18  6 19  0 11  6 12 19  1  8  6 18  5 11 10 17  1 12 14
 11 10 18  1 12  5 17 11  1 14 10 15  5 12  3  8 11 10 14  3  9 13 12 15
  9  8 10 13 16  2 17  3  8 17 15 18  9 13  3 16  2  9 18 15  3  2 17  9
 13  2 13  2]
Root:  17af Voisinage:  8960  ,makeSpan:  1267 / 1267  ,écart:  0.0
[ 4 14 15  6 19  7  0  5 16 19  0  4  1  7  4 16  5  0  4  7  4 16  7  6
 19  0 14  7  8 18  6 19  0 11  6 12 19  1  8  6 18  5 11 10 17  1 12 14
 11 10 18  1 12  5 17 11  1 14 10 15  